Obtain data form Coinranking API   

In [1]:
import requests,json
import pandas as pd

coin_id = 1 # 1 is BTC, 2 is ETH, 4 is BCH
time_range = "1y"
price_currency = "EUR"
response = requests.get("https://api.coinranking.com/v1/public/coin/%s/history/%s?base=%s" % (coin_id, time_range, price_currency))
response

<Response [200]>

Dump response to JSON file

In [2]:
import os

os.makedirs('responses', exist_ok=True)
file = open('responses/coin%d-history%s-currency%s.json' % (coin_id, time_range, price_currency),'w+')
file.write(response.text)
file.close()


Load response in pandas data frame

In [3]:
response_json = json.loads(response.text)['data']['history']
data_frame = pd.DataFrame(response_json)
data_frame['price'] = pd.to_numeric(data_frame['price'])
data_frame['timestamp'] = pd.to_datetime(data_frame['timestamp'], unit='ms').dt.date
data_sequence = data_frame.groupby('timestamp').mean()['price']
data_sequence

timestamp
2018-09-02    6115.234563
2018-09-03    6237.371389
2018-09-04    6253.482447
2018-09-05    6324.652042
2018-09-06    6152.730976
                 ...     
2019-08-28    9059.787395
2019-08-29    8616.555789
2019-08-30    8667.161586
2019-08-31    8742.433461
2019-09-01    8716.940672
Name: price, Length: 365, dtype: float64

Normalize data

In [6]:
chunk_size = 30
chunks = []
for i in range(len(data_sequence) - chunk_size+1):
    chunks.append(data_sequence[i:i+chunk_size])

price_change_sequence = []
for chunk in chunks:
    # TODO: Rethink technique, add float casting
    price_change_sequence.append([(price / chunk[0]) - 1 for price in chunk])
price_change_sequence

Split data into training and test sets

In [6]:
import numpy as np

price_matrix = np.array(price_change_sequence)
train_set_size = int(len(price_matrix) * 0.8)
train_set = price_matrix[:train_set_size, :]
np.random.shuffle(train_set)

Error evaluating: thread_id: pid_4576_id_4448772304
frame_id: 4556359312
scope: FRAME
attrs: __py_debug_temp_var_793658317
Traceback (most recent call last):
  File "/Users/Pomeranek/Library/Application Support/JetBrains/Toolbox/apps/PyCharm-P/ch-0/192.5728.105/PyCharm.app/Contents/helpers/pydev/_pydevd_bundle/pydevd_vars.py", line 285, in resolve_compound_variable_fields
    return _typeName, resolver.get_dictionary(VariableWithOffset(var, offset) if offset else var)
  File "/Users/Pomeranek/Library/Application Support/JetBrains/Toolbox/apps/PyCharm-P/ch-0/192.5728.105/PyCharm.app/Contents/helpers/pydev/pydevd_plugins/extensions/types/pydevd_plugin_numpy_types.py", line 91, in get_dictionary
    ret['min'] = obj.min()
  File "/usr/local/lib/python3.7/site-packages/numpy/core/_methods.py", line 29, in _amin
    return umr_minimum(a, axis, None, out, keepdims)
ValueError: zero-size array to reduction operation minimum which has no identity
